In [211]:
import numpy as np
import pandas as pd

#### We have $M$ states and $K$ possible emissions named as $p_M$ and $e_K$.
#### We also have a sequence of length L of numbers $e_i$ as I understand.

#### Firstly, I want to generate emission and transition matrices (row-stochastic as I understand) and begin distribution.

In [320]:
# Ask user to enter the dimension of states, emissions and sequence.
M = int(input('Please enter number of states:'))
K = int(input('Please enter number of emissions:'))
L = int(input('Please enter length of sequence:'))
# Generate random matrices with index and columns named 'p_i','e_j'
transition = pd.DataFrame(np.random.randint(0,10,size=(M, M))).add_prefix('p_').T.add_prefix('p_').T
emission = pd.DataFrame(np.random.randint(0,10,size=(M, K))).add_prefix('e_').T.add_prefix('p_').T
begin_distribution = pd.DataFrame(np.random.randint(1,10,size=(M, 1))).T.add_prefix('p_').T
# Normalize matrices to have row-stochastic
transition = transition.div(transition.sum(axis=1),axis=0)
emission = emission.div(emission.sum(axis=1),axis=0)
begin_distribution = begin_distribution.div(begin_distribution.sum(axis=0),axis=1)
print('Random emission and transition matrices and begin distribution are created')
#Save matrices
transition.to_csv('transition.csv',sep='\t')
emission.to_csv('emission.csv',sep='\t')
begin_distribution.to_csv('begin_distribution.csv',sep='\t')
print('Generated tables were saved')

Please enter number of states:10
Please enter number of emissions:6
Please enter length of sequence:5
Random emission and transition matrices and begin distribution are created
Generated tables were saved


#### Now we read given matrices and distribution with tab-separator.

In [321]:
#Read given matrices with tab-separator
transition1 = pd.read_csv('transition.csv',sep='\t',index_col=0)
emission1 = pd.read_csv('emission.csv',sep='\t',index_col=0)
begin_distribution1 = pd.read_csv('begin_distribution.csv',sep='\t',index_col=0)

#### After reading the transition matrix looks like:

In [322]:
transition1

,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9
p_0,0.085106,0.191489,0.106383,0.191489,0.042553,0.170213,0.085106,0.042553,0.063830,0.021277
p_1,0.097561,0.195122,0.073171,0.121951,0.170732,0.024390,0.195122,0.024390,0.073171,0.024390
p_2,0.085106,0.085106,0.191489,0.148936,0.106383,0.042553,0.042553,0.191489,0.021277,0.085106
p_3,0.062500,0.187500,0.125000,0.093750,0.093750,0.000000,0.000000,0.250000,0.093750,0.093750
p_4,0.076923,0.038462,0.000000,0.269231,0.192308,0.153846,0.000000,0.192308,0.000000,0.076923
p_5,0.200000,0.100000,0.050000,0.125000,0.000000,0.025000,0.150000,0.200000,0.075000,0.075000
p_6,0.000000,0.114286,0.028571,0.057143,0.257143,0.000000,0.142857,0.114286,0.200000,0.085714
p_7,0.065217,0.086957,0.108696,0.152174,0.195652,0.021739,0.173913,0.108696,0.065217,0.021739
p_8,0.200000,0.175000,0.075000,0.125000,0.000000,0.025000,0.125000,0.050000,0.225000,0.000000
p_9,0.159091,0.090909,0.045455,0.090909,0.068182,0.204545,0.204545,0.090909,0.022727,0.022727


#### Then we need to generate some sequence of length $L$ of alphabet $e_k$

In [323]:
# select some state for generating simulated path
def random_choice(distribution,p):
    s = np.random.random()*distribution[p].sum()
    for i in range(len(distribution)):
        s = s - distribution[p]['p_'+str(i)]
        if s < 0:
            return 'p_'+str(i)

# select some emission for generating sequence        
def random_choice2(distribution,p):
    s = np.random.random()*distribution[p].sum()
    for i in range(len(distribution)):
        s = s - distribution[p]['e_'+str(i)]
        if s < 0:
            return 'e_'+str(i)

#function of generating simulated path
def HMM_random_generator_path(begin_distribution1,transition1,L):
    path = []
    path.append(random_choice(begin_distribution1,'0'))
    for i in range(1,L):
        path.append(random_choice(pd.DataFrame(transition1.T[path[i-1]]),path[i-1]))
    return path

#function of generating sequence
def HMM_random_generator_sequence(states,emission1,L):
    sequence = []
    for i in range(L):
        sequence.append(random_choice2(pd.DataFrame(emission1.T[states[i]]),states[i]))
    return sequence

In [324]:
#generating path and sequence
simulated_path = HMM_random_generator_path(begin_distribution1,transition1,L)
sequence = HMM_random_generator_sequence(simulated_path,emission1,L)

In [325]:
simulated_path

['p_8', 'p_6', 'p_8', 'p_8', 'p_6']

In [326]:
sequence

['e_4', 'e_2', 'e_2', 'e_0', 'e_0']

### Forward Algorithm

In [327]:
#function for forward algorithm
def forward_propagation(begin_distribution1,transition1,emission1,sequence):
    F = np.zeros((L,M))
    F[0] = begin_distribution1['0']*emission1[sequence[0]]
    for i in range(1,L):
        F[i] = F[i-1]@transition1*emission1[sequence[i]]
    return pd.DataFrame(F).add_prefix('p_').T

#function for calculating P(x)
def P(data,L):
    return data[L-1].sum()

In [328]:
F = forward_propagation(begin_distribution1,transition1,emission1,sequence)
F

,0,1,2,3,4
p_0,0.020860,0.003931,0.000678,0.000132,0.000016
p_1,0.000770,0.003899,0.000878,0.000170,0.000025
p_2,0.011299,0.003742,0.000925,0.000020,0.000003
p_3,0.030618,0.007246,0.001223,0.000212,0.000025
p_4,0.040678,0.003536,0.000741,0.000000,0.000000
p_5,0.035682,0.001862,0.000302,0.000000,0.000000
p_6,0.014528,0.002503,0.000575,0.000029,0.000005
p_7,0.005260,0.001991,0.000322,0.000186,0.000020
p_8,0.019136,0.004276,0.000887,0.000059,0.000007
p_9,0.000000,0.001206,0.000197,0.000058,0.000006


#### For our sequence let's calculate P(x)

In [330]:
print ('P(x)=',P(F,L))

P(x)= 0.00010612766454657125


### Backward Algorithm

In [279]:
#function for backward algorithm
def backward_propagation(F,transition1,emission1,sequence1):
    F1 = pd.DataFrame(np.zeros((M,L)),index=transition1.index)
    F1[len(sequence)-1]=F[len(sequence)-1]
    for j in range(len(sequence)-2,-1,-1):
        for i in F1.index:
            p_sum = 0
            for k in F1.index:
                p_sum += F1[j+1][k]*transition1[k][i]*emission1[sequence1[j+1]][k]
            F1[j][i] = p_sum
    return F1

In [280]:
backward_propagation(F,transition1,emission1,sequence)

,0,1,2,3,4
p_0,1.338267e-08,8.203740e-08,4.839560e-07,0.000002,0.000000
p_1,1.066438e-08,6.454379e-08,3.900008e-07,0.000002,0.000003
p_2,1.047285e-08,5.745313e-08,4.787047e-07,0.000003,0.000006
p_3,1.208448e-08,6.339686e-08,4.919553e-07,0.000003,0.000009
p_4,1.349240e-08,8.183399e-08,3.748705e-07,0.000002,0.000003
p_5,1.268800e-08,6.211457e-08,5.337845e-07,0.000003,0.000003
p_6,1.326637e-08,6.910310e-08,4.188851e-07,0.000003,0.000028
p_7,1.286970e-08,6.943657e-08,6.180503e-07,0.000003,0.000020
p_8,1.407913e-08,8.153781e-08,3.047299e-07,0.000002,0.000017
p_9,1.152001e-08,7.769806e-08,4.963827e-07,0.000002,0.000012


### Viterbi Algorithm

In [281]:
# additional function for getting maximum element in sum of probabilities
def get_max(values):
    maxv = values[0]
    maxi = 0
    for ind, val in enumerate(values):
        if val>maxv:
            maxv = val
            maxi = ind
    return maxv,maxi

In [331]:
#viterbi
def viterbi_algorithm(transition1,emission1,sequence1):
    F = pd.DataFrame(np.zeros((M,L+1)),index=transition1.index)
    F[0]=begin_distribution1['0']
    c = pd.DataFrame(np.zeros((M,L+1)),index=transition1.index)
    for k in F.columns[1:]:
        for i in F.index:
            values = []
            for j in F.index:
                values.append(F[k-1][j]*transition1[i][j]*emission1[sequence1[k-1]][i])
            maxv, maxi = get_max(values)
            F[k][i] = maxv
            c[k][i] = maxi
    return F, c

In [332]:
F, c = viterbi_algorithm(transition1,emission1,sequence)

In [333]:
F

,0,1,2,3,4,5
p_0,0.135593,0.004172,0.000249,0.000014,8.845969e-07,1.926138e-08
p_1,0.016949,0.001180,0.000321,0.000021,8.156703e-07,3.079834e-08
p_2,0.084746,0.002211,0.000353,0.000023,1.489032e-07,4.483945e-09
p_3,0.118644,0.009421,0.000622,0.000024,8.052130e-07,4.252515e-08
p_4,0.135593,0.008954,0.000344,0.000013,0.000000e+00,0.000000e+00
p_5,0.135593,0.006074,0.000217,0.000008,0.000000e+00,0.000000e+00
p_6,0.067797,0.005685,0.000174,0.000012,1.970493e-07,1.024902e-08
p_7,0.152542,0.001023,0.000162,0.000011,1.237569e-06,4.164895e-08
p_8,0.084746,0.004306,0.000330,0.000022,4.694962e-07,1.022290e-08
p_9,0.067797,0.000000,0.000093,0.000006,3.541726e-07,1.191927e-08


In [334]:
c

,0,1,2,3,4,5
p_0,0.0,5.0,5.0,8.0,8.0,8.0
p_1,0.0,0.0,3.0,3.0,3.0,0.0
p_2,0.0,7.0,3.0,3.0,2.0,7.0
p_3,0.0,4.0,4.0,4.0,4.0,7.0
p_4,0.0,7.0,4.0,4.0,0.0,0.0
p_5,0.0,0.0,4.0,4.0,0.0,0.0
p_6,0.0,7.0,5.0,1.0,1.0,7.0
p_7,0.0,3.0,3.0,3.0,3.0,3.0
p_8,0.0,8.0,6.0,8.0,8.0,8.0
p_9,0.0,0.0,3.0,3.0,3.0,3.0


In [335]:
# let's see the path of Viterbi
def decoding(F,c):
    path = []
    idx = F[L].idxmax(axis=0)
    path = [idx]+path
    k = 'p_'+str(c[L][idx]).split('.')[0]
    path = [k] + path 
    for i in range(L-1,1,-1):
        k = 'p_'+ str(c[i][k]).split('.')[0]
        path = [k] + path
    return path

In [338]:
decoding(F,c)

['p_4', 'p_4', 'p_3', 'p_7', 'p_3']

In [339]:
simulated_path

['p_8', 'p_6', 'p_8', 'p_8', 'p_6']

### Unfair casino

#### $p_0$ corresponds to fair dice and $p_1$ for loaded dice
#### $e_0,...,e_5$  corresponds to numbers 1...6 in dice

In [340]:
M = int(input('Please enter number of states:')) #here you must enter 2
K = int(input('Please enter number of emissions:')) # here you must enter 6
L = int(input('Please enter length of sequence:'))
transition = pd.DataFrame(np.random.randint(0,10,size=(M, M))).add_prefix('p_').T.add_prefix('p_').T
emission = pd.DataFrame(np.random.randint(0,10,size=(M, K))).add_prefix('e_').T.add_prefix('p_').T
begin_distribution = pd.DataFrame(np.random.randint(1,10,size=(M, 1))).T.add_prefix('p_').T
transition = transition.div(transition.sum(axis=1),axis=0)
emission = emission.div(emission.sum(axis=1),axis=0)
begin_distribution = begin_distribution.div(begin_distribution.sum(axis=0),axis=1)
print('Random emission and transition matrices and begin distribution are created')
transition.to_csv('transition.csv',sep='\t')
emission.to_csv('emission.csv',sep='\t')
begin_distribution.to_csv('begin_distribution.csv',sep='\t')
print('Generated tables were saved')

Please enter number of states:2
Please enter number of emissions:6
Please enter length of sequence:20
Random emission and transition matrices and begin distribution are created
Generated tables were saved


In [341]:
transition1 = pd.read_csv('transition.csv',sep='\t',index_col=0)
emission1 = pd.read_csv('emission.csv',sep='\t',index_col=0)
begin_distribution1 = pd.read_csv('begin_distribution.csv',sep='\t',index_col=0)

In [342]:
## our data for casino
transition1['p_0']['p_0']=0.95
transition1['p_0']['p_1']=0.1
transition1['p_1']['p_0']=0.05
transition1['p_1']['p_1']=0.9
begin_distribution1['0']['p_0']=2/3
begin_distribution1['0']['p_1']=1/3
emission1['e_0']['p_0']=1/6
emission1['e_1']['p_0']=1/6
emission1['e_2']['p_0']=1/6
emission1['e_3']['p_0']=1/6
emission1['e_4']['p_0']=1/6
emission1['e_5']['p_0']=1/6
emission1['e_0']['p_1']=0.1
emission1['e_1']['p_1']=0.1
emission1['e_2']['p_1']=0.1
emission1['e_3']['p_1']=0.1
emission1['e_4']['p_1']=0.1
emission1['e_5']['p_1']=0.5

In [343]:
#simulate some path and sequence
simulated_path = HMM_random_generator_path(begin_distribution1,transition1,L)
sequence = HMM_random_generator_sequence(simulated_path,emission1,L)

In [344]:
F = forward_propagation(begin_distribution1,transition1,emission1,sequence)
B = backward_propagation(F,transition1,emission1,sequence)

In [345]:
#Viterbi
F, c = viterbi_algorithm(transition1,emission1,sequence)

In [346]:
decoding(F,c)

['p_0',
 'p_0',
 'p_0',
 'p_0',
 'p_0',
 'p_0',
 'p_0',
 'p_0',
 'p_0',
 'p_0',
 'p_0',
 'p_0',
 'p_0',
 'p_0',
 'p_0',
 'p_0',
 'p_0',
 'p_0',
 'p_0',
 'p_0']

In [347]:
simulated_path

['p_1',
 'p_1',
 'p_1',
 'p_1',
 'p_1',
 'p_0',
 'p_0',
 'p_0',
 'p_0',
 'p_0',
 'p_0',
 'p_1',
 'p_0',
 'p_0',
 'p_0',
 'p_0',
 'p_0',
 'p_0',
 'p_0',
 'p_0']